# AutoCron Demo 4: Task Persistence

This notebook demonstrates how to save and restore tasks across system restarts.

## Features Covered:
- Saving tasks to YAML files
- Loading tasks from files
- Task state persistence
- Backup and restore workflows
- Configuration management

## Setup

In [ ]:
from autocron import AutoCron
from datetime import datetime
import os
from pathlib import Path

## 1. Basic Task Persistence

Save tasks to a file and load them back.

In [ ]:
# Create scheduler and add tasks
scheduler = AutoCron()

def backup_database():
    print(f"[{datetime.now().strftime('%H:%M:%S')}] 💾 Backing up database...")
    return "Backup complete"

def send_report():
    print(f"[{datetime.now().strftime('%H:%M:%S')}] 📊 Sending report...")
    return "Report sent"

def cleanup_logs():
    print(f"[{datetime.now().strftime('%H:%M:%S')}] 🧹 Cleaning up logs...")
    return "Logs cleaned"

# Add tasks
scheduler.add_task(name="database_backup", func=backup_database, every='6h')
scheduler.add_task(name="daily_report", func=send_report, cron='0 9 * * *')
scheduler.add_task(name="log_cleanup", func=cleanup_logs, every='1d')

print(f"✅ Added {len(scheduler.tasks)} tasks to scheduler\n")
for task in scheduler.tasks:
    print(f"  📋 {task.name} - Schedule: {task.schedule}")

In [ ]:
# Save tasks to file
save_path = "tasks_config.yaml"
scheduler.save_tasks(save_path)

print(f"💾 Tasks saved to: {save_path}")
print(f"📁 File size: {os.path.getsize(save_path)} bytes")

In [ ]:
# View the saved YAML file
with open(save_path, 'r') as f:
    content = f.read()
    print("📄 Saved YAML Content:\n")
    print(content)

In [ ]:
# Create a new scheduler and load tasks
new_scheduler = AutoCron()

print(f"Before loading: {len(new_scheduler.tasks)} tasks\n")

new_scheduler.load_tasks(save_path)

print(f"After loading: {len(new_scheduler.tasks)} tasks\n")
for task in new_scheduler.tasks:
    print(f"  ✅ Loaded: {task.name} - Schedule: {task.schedule}")

## 2. Default Save Location

Tasks are automatically saved to `~/.autocron/tasks.yaml` by default.

In [ ]:
scheduler = AutoCron()

scheduler.add_task(
    name="health_check",
    func=lambda: print("Health check"),
    every='5m'
)

# Save to default location
scheduler.save_tasks()  # No path specified

default_path = Path.home() / ".autocron" / "tasks.yaml"
print("💾 Tasks saved to default location:")
print(f"📁 {default_path}")
print(f"✅ Exists: {default_path.exists()}")

## 3. Task Metadata Persistence

Save tasks with all their configuration including metadata, tags, and priorities.

In [ ]:
scheduler = AutoCron()

# Add task with full configuration
scheduler.add_task(
    name="etl_pipeline",
    func=lambda: print("Running ETL"),
    every='1h',
    priority=10,
    retries=3,
    retry_delay=60,
    timeout=300,
    max_instances=1,
    enabled=True,
    tags=["etl", "critical", "production"],
    metadata={
        "owner": "Data Team",
        "version": "2.1",
        "cost_per_run": 0.05,
        "sla_minutes": 30
    }
)

# Save with all configuration
config_path = "full_config.yaml"
scheduler.save_tasks(config_path)

print(f"💾 Task saved with full configuration\n")

# View the saved configuration
with open(config_path, 'r') as f:
    print("📄 Full Configuration:\n")
    print(f.read())

In [ ]:
# Load and verify all configuration is preserved
restored_scheduler = AutoCron()
restored_scheduler.load_tasks(config_path)

task = restored_scheduler.tasks[0]
print("✅ Restored Task Configuration:\n")
print(f"  Name: {task.name}")
print(f"  Schedule: {task.schedule}")
print(f"  Priority: {task.priority}")
print(f"  Retries: {task.retries}")
print(f"  Timeout: {task.timeout}s")
print(f"  Tags: {task.tags}")
print(f"  Metadata:")
for key, value in task.metadata.items():
    print(f"    - {key}: {value}")

## 4. Multiple Configuration Files

Manage different task sets for different environments.

In [ ]:
# Development tasks
dev_scheduler = AutoCron()
dev_scheduler.add_task(name="dev_test", func=lambda: print("Dev"), every='1m')
dev_scheduler.add_task(name="debug_log", func=lambda: print("Debug"), every='30s')
dev_scheduler.save_tasks("tasks_development.yaml")

# Staging tasks
staging_scheduler = AutoCron()
staging_scheduler.add_task(name="staging_test", func=lambda: print("Staging"), every='5m')
staging_scheduler.add_task(name="integration_test", func=lambda: print("Integration"), every='15m')
staging_scheduler.save_tasks("tasks_staging.yaml")

# Production tasks
prod_scheduler = AutoCron()
prod_scheduler.add_task(name="prod_backup", func=lambda: print("Backup"), every='1h')
prod_scheduler.add_task(name="health_monitor", func=lambda: print("Health"), every='30s')
prod_scheduler.save_tasks("tasks_production.yaml")

print("✅ Saved tasks for 3 environments:\n")
print(f"  📁 tasks_development.yaml - {len(dev_scheduler.tasks)} tasks")
print(f"  📁 tasks_staging.yaml - {len(staging_scheduler.tasks)} tasks")
print(f"  📁 tasks_production.yaml - {len(prod_scheduler.tasks)} tasks")

In [ ]:
# Load environment-specific tasks
import os

# Simulate environment variable
environment = "production"  # or "development", "staging"

config_file = f"tasks_{environment}.yaml"
scheduler = AutoCron()
scheduler.load_tasks(config_file)

print(f"🚀 Loaded {environment.upper()} configuration")
print(f"📋 Tasks loaded: {len(scheduler.tasks)}\n")
for task in scheduler.tasks:
    print(f"  ✅ {task.name}")

## 5. Backup and Restore Workflow

Create timestamped backups of your task configurations.

In [ ]:
from datetime import datetime
from pathlib import Path

# Create scheduler with tasks
scheduler = AutoCron()
scheduler.add_task(name="task1", func=lambda: None, every='1h')
scheduler.add_task(name="task2", func=lambda: None, every='30m')
scheduler.add_task(name="task3", func=lambda: None, cron='0 9 * * *')

# Create backups directory
backup_dir = Path("backups")
backup_dir.mkdir(exist_ok=True)

# Save with timestamp
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
backup_file = backup_dir / f"tasks_backup_{timestamp}.yaml"
scheduler.save_tasks(str(backup_file))

print("💾 Backup created:")
print(f"📁 {backup_file}")
print(f"📊 Size: {backup_file.stat().st_size} bytes")
print(f"⏰ Timestamp: {timestamp}")

In [ ]:
# List all backups
backup_dir = Path("backups")
backups = sorted(backup_dir.glob("tasks_backup_*.yaml"), reverse=True)

print("📦 Available Backups:\n")
for i, backup in enumerate(backups, 1):
    size = backup.stat().st_size
    mtime = datetime.fromtimestamp(backup.stat().st_mtime)
    print(f"{i}. {backup.name}")
    print(f"   Size: {size} bytes")
    print(f"   Modified: {mtime.strftime('%Y-%m-%d %H:%M:%S')}\n")

In [ ]:
# Restore from latest backup
if backups:
    latest_backup = backups[0]
    
    restored_scheduler = AutoCron()
    restored_scheduler.load_tasks(str(latest_backup))
    
    print(f"♻️  Restored from: {latest_backup.name}")
    print(f"📋 Tasks restored: {len(restored_scheduler.tasks)}\n")
    for task in restored_scheduler.tasks:
        print(f"  ✅ {task.name}")
else:
    print("❌ No backups found")

## 6. Persistence in Production

Best practices for using task persistence in production.

In [ ]:
import sys
from pathlib import Path

class ProductionScheduler:
    """Production-ready scheduler with automatic persistence."""
    
    def __init__(self, config_file="tasks.yaml"):
        self.scheduler = AutoCron()
        self.config_file = Path(config_file)
        
        # Create config directory if needed
        self.config_file.parent.mkdir(parents=True, exist_ok=True)
        
        print(f"🚀 Production Scheduler initialized")
        print(f"📁 Config file: {self.config_file}")
    
    def load_or_create(self):
        """Load existing tasks or start fresh."""
        if self.config_file.exists():
            print(f"\n📂 Loading existing configuration...")
            self.scheduler.load_tasks(str(self.config_file))
            print(f"✅ Loaded {len(self.scheduler.tasks)} tasks")
        else:
            print(f"\n🆕 No existing configuration found")
            print(f"📝 Starting with empty scheduler")
        return self
    
    def add_task(self, **kwargs):
        """Add task and auto-save."""
        self.scheduler.add_task(**kwargs)
        self.save()
        print(f"✅ Task '{kwargs['name']}' added and saved")
    
    def save(self):
        """Save current configuration."""
        self.scheduler.save_tasks(str(self.config_file))
    
    def start(self):
        """Start the scheduler."""
        print(f"\n🚀 Starting scheduler with {len(self.scheduler.tasks)} tasks...\n")
        try:
            self.scheduler.start()
        except KeyboardInterrupt:
            print("\n⏹️  Scheduler stopped by user")
        except Exception as e:
            print(f"\n❌ Error: {e}")
        finally:
            self.save()
            print("💾 Configuration saved")

# Example usage
prod = ProductionScheduler("production_tasks.yaml")
prod.load_or_create()

# Add tasks (will be auto-saved)
prod.add_task(
    name="monitor",
    func=lambda: print("Monitoring..."),
    every='1m'
)

print("\n✅ Production scheduler ready!")
print("📝 All changes are automatically persisted")

## 7. Task Migration

Migrate tasks between different schedulers or systems.

In [ ]:
# Export from old system
old_scheduler = AutoCron()
old_scheduler.add_task(name="legacy_task_1", func=lambda: None, every='1h')
old_scheduler.add_task(name="legacy_task_2", func=lambda: None, every='30m')
old_scheduler.save_tasks("legacy_export.yaml")

print("📤 Exported from old system: 2 tasks\n")

# Import to new system
new_scheduler = AutoCron()
new_scheduler.load_tasks("legacy_export.yaml")

# Add new tasks
new_scheduler.add_task(name="new_task_1", func=lambda: None, every='15m')
new_scheduler.add_task(name="new_task_2", func=lambda: None, cron='0 * * * *')

# Save combined configuration
new_scheduler.save_tasks("migrated_tasks.yaml")

print("📥 Imported to new system")
print(f"📊 Total tasks after migration: {len(new_scheduler.tasks)}\n")
for task in new_scheduler.tasks:
    prefix = "📦" if "legacy" in task.name else "🆕"
    print(f"  {prefix} {task.name}")

## 8. Real-World Example: Server Restart Recovery

Automatically restore tasks after server restarts.

In [ ]:
#!/usr/bin/env python3
"""
server_startup.py - Auto-load tasks on server start
"""

from autocron import AutoCron
from pathlib import Path
import logging

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

def initialize_scheduler():
    """Initialize scheduler with persisted tasks."""
    config_file = Path("/etc/autocron/tasks.yaml")  # System config location
    
    scheduler = AutoCron()
    
    if config_file.exists():
        logger.info(f"Loading tasks from {config_file}")
        try:
            scheduler.load_tasks(str(config_file))
            logger.info(f"Successfully loaded {len(scheduler.tasks)} tasks")
            
            # Log loaded tasks
            for task in scheduler.tasks:
                logger.info(f"  - {task.name}: {task.schedule}")
            
        except Exception as e:
            logger.error(f"Failed to load tasks: {e}")
            return None
    else:
        logger.warning(f"No configuration file found at {config_file}")
        logger.info("Starting with empty scheduler")
    
    return scheduler

# Simulate server startup
print("🖥️  Server Starting...\n")
print("="*50)

if scheduler := initialize_scheduler():
    print("\n✅ Scheduler initialized successfully")
    print(f"📊 Ready to execute {len(scheduler.tasks)} tasks")
    print("\n💡 Tasks will survive server restarts!")
else:
    print("\n❌ Failed to initialize scheduler")

## Cleanup

Remove demo files created in this notebook.

In [ ]:
import os
import shutil

# Remove individual files
demo_files = [
    "tasks_config.yaml",
    "full_config.yaml",
    "tasks_development.yaml",
    "tasks_staging.yaml",
    "tasks_production.yaml",
    "production_tasks.yaml",
    "legacy_export.yaml",
    "migrated_tasks.yaml"
]

for file in demo_files:
    if os.path.exists(file):
        os.remove(file)
        print(f"🗑️  Removed: {file}")

# Remove backups directory
if os.path.exists("backups"):
    shutil.rmtree("backups")
    print("🗑️  Removed: backups/")

print("\n✅ Cleanup complete!")

## Summary

In this demo, you learned:

✅ **Save Tasks** - Export task configurations to YAML files

✅ **Load Tasks** - Restore tasks from saved configurations

✅ **Full Configuration** - Persist all task settings (metadata, tags, priorities)

✅ **Multiple Environments** - Manage dev, staging, and production configs

✅ **Backup & Restore** - Create timestamped backups for safety

✅ **Production Patterns** - Auto-save, auto-load, server restart recovery

✅ **Task Migration** - Move tasks between systems

### Best Practices:

1. **Version Control** - Store task configs in Git
2. **Environment-Specific** - Separate configs for dev/staging/prod
3. **Regular Backups** - Automated timestamped backups
4. **Auto-Save** - Save after every task modification
5. **Startup Recovery** - Auto-load on server/application start

### Use Cases:

- 🔄 **Server Restarts** - Tasks survive reboots
- 🚀 **Deployments** - Maintain tasks across updates
- 👥 **Team Sharing** - Share task configs via Git
- 🌍 **Multi-Environment** - Separate configs per environment
- 📦 **Backup/Restore** - Disaster recovery

### Next Steps:
- Check out `05_safe_mode.ipynb` for secure task execution
- See `06_dashboard.ipynb` for visual monitoring
- Explore `07_notifications.ipynb` for alerts